In [ ]:
import lightgbm as lgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
!pip install optuna

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append('/content/drive/MyDrive/Freshman/UROP/Transformer')

import data_preprocessing

Mounted at /content/drive


In [ ]:
data_raw = pd.read_csv('/content/drive/MyDrive/Freshman/UROP/fraud_detection-main/credit_card_transactions.csv')

train_data_raw = data_raw.sample(frac=0.2, random_state=42)
test_data_raw = data_raw.drop(train_data_raw.index)

print(train_data_raw.shape)
print(test_data_raw.shape)

(259335, 24)
(1037340, 24)


Data Preprocessing

In [ ]:
from data_preprocessing import data_preprocessing

train_data_processed, test_data_processed = data_preprocessing(train_data_raw.sort_values(by = ["cc_num"])[:2500], test_data_raw.sort_values(by = ["cc_num"])[:20000])

train_data_processed = train_data_processed[train_data_processed['is_fraud'] == 0]

In [ ]:
from sklearn.preprocessing import StandardScaler

train_data = train_data_processed.drop(['lat', 'long', 'city_pop', 'state', 'gender', 'Year'], axis=1)
test_data = test_data_processed.drop(['lat', 'long', 'city_pop', 'state', 'gender', 'Year'], axis=1)

for feature in ['amt', 'merch_lat', 'merch_long', 'LatLong_Dist']:
  train_data[feature] = StandardScaler().fit_transform(train_data[feature].values.reshape(-1, 1))
  test_data[feature] = StandardScaler().fit_transform(test_data[feature].values.reshape(-1, 1))

for feature in ['category', 'Hour', 'Minute', 'Second', 'dayOfWeek']:
  train_data[feature] += 1
  test_data[feature] += 1

column_order = ['cc_num', 'is_fraud', 'category', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'dayOfWeek', 'amt', 'merch_lat', 'merch_long', 'LatLong_Dist']
train_data = train_data[column_order]
test_data = test_data[column_order]

In [ ]:
train_data.head()

,cc_num,is_fraud,category,Month,Day,Hour,Minute,Second,dayOfWeek,amt,merch_lat,merch_long,LatLong_Dist
1017,60416207185,0,1,1,1,13,48,16,2,-0.404749,1.599109,-1.053304,1.750297
2907,60416207185,0,2,1,2,14,11,47,3,-0.255188,1.439331,-0.940773,-0.106327
7473,60416207185,0,3,1,5,22,35,21,6,-0.421951,1.392894,-0.996827,-2.192852
10739,60416207185,0,4,1,7,13,59,20,1,1.074185,1.125739,-0.963345,1.280002
14749,60416207185,0,5,1,9,10,2,11,3,0.114020,1.538082,-1.004556,0.122997


Transformer Class

In [ ]:
!pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    # the model dimension d_model must be divisible by num_heads
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads
    self.d_k = d_model // num_heads # (d_model * num_features // num_heads)

    self.W_q = nn.Linear(d_model, d_model) # Query transformation
    self.W_k = nn.Linear(d_model, d_model) # Key transformation
    self.W_v = nn.Linear(d_model, d_model) # Value transformation
    self.W_o = nn.Linear(d_model, d_model) # Output transformation

  def scaled_dot_product_attention(self, Q, K, V, mask=None):
    # Calculate attention scores
    attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k) # (batch_size, num_heads, seq_length, seq_length)
    if mask is not None:
      attn_scores = attn_scores.masked_fill(mask == 0, -1e9) # (batch_size, num_heads, seq_length, seq_length)
    attn_probs = torch.softmax(attn_scores, dim=-1) # (batch_size, num_heads, seq_length, seq_length)
    output = torch.matmul(attn_probs, V) # (batch_size, num_heads, seq_length, d_k)
    return output

  def split_heads(self, x):
    # Reshape the input to have num_heads for multi-head attention
    batch_size, seq_length, d_model = x.size()
    return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

  def combine_heads(self, x):
    # Combine the multiple heads back to original shape
    batch_size, _, seq_length, d_k = x.size()
    return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

  def forward(self, Q, K, V, mask=None):
    Q = self.split_heads(self.W_q(Q)) # (batch_size, num_heads, seq_length, d_k)
    K = self.split_heads(self.W_k(K))
    V = self.split_heads(self.W_v(V))

    attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

    output = self.W_o(self.combine_heads(attn_output)) # (batch_size, seq_length, d_model*num_features)
    return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff):
    super(PositionWiseFeedForward, self).__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()
    self.self_attn = MultiHeadAttention(d_model, num_heads)
    self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, tgt_mask):
    attn_output = self.self_attn(x, x, x, tgt_mask)
    x = self.norm1(x + self.dropout(attn_output))
    ff_output = self.feed_forward(x)
    x = self.norm2(x + self.dropout(attn_output))
    return x

In [ ]:
class Transformer(nn.Module):
  def __init__(self, categorical, numeric, num_layers, max_sequence_length, d_model=64, field_depth=1, num_heads=4, dim_feedforward=128, dropout=0.1):
    # d_model is the dimensionality of the input
    # categorical is the tuple of sizes of each categorical field
    # numeric is the number of numeric fields
    super(Transformer, self).__init__()
    self.categorical = categorical
    self.numeric = numeric

    self.num_categories = len(categorical)
    self.num_numeric = numeric
    self.total_fields = self.num_categories + self.num_numeric

    self.categorical_embedding = nn.ModuleList([nn.Embedding(c, d_model, padding_idx=0) for c in categorical])
    self.numeric_embedding = nn.ModuleList([nn.Linear(1, d_model) for _ in range(numeric)])
    self.positional_embedding = nn.Embedding(max_sequence_length, d_model * self.total_fields) # learned positional embedding

    self.decoder_layers = nn.ModuleList([DecoderLayer(d_model*self.total_fields, num_heads, dim_feedforward, dropout) for _ in range(num_layers)])

    self.cat_output_heads = nn.ModuleList([nn.Linear(d_model*self.total_fields, c) for c in categorical])
    self.num_output_heads = nn.ModuleList([nn.Linear(d_model*self.total_fields, 1) for _ in range(numeric)])
    self.embed_pred = nn.Linear(d_model * self.total_fields, d_model * self.total_fields)

    self.dropout = nn.Dropout(dropout)

  def generate_mask(self, tgt):
    # returning nopeak mask instead of src, tgt, commented out unsqueeze
    tgt_mask = (tgt[:, 1] != 0).unsqueeze(0).unsqueeze(1).unsqueeze(3)
    seq_length = tgt.size(0)
    nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    return nopeak_mask

  def forward(self, tgt):
    # categorical: Tensor of category indices
    # numeric: tensor of numeric values
    # return predicted embedding of next transaction for each categorical field

    tgt_mask = self.generate_mask(tgt)

    tgt_categorical = tgt[:, :self.num_categories].long()
    tgt_numeric = tgt[:, self.num_categories:self.num_categories + self.num_numeric + 1].float()

    tgt_categorical_embedded = torch.cat([self.categorical_embedding[i](tgt_categorical[:, i]) for i in range(self.num_categories)], dim=1) # (seq_length, num_categories*d_model)
    tgt_numeric_embedded = torch.cat([self.numeric_embedding[i](tgt_numeric[:, i].unsqueeze(-1)) for i in range(self.num_numeric)], dim=1) # (seq_length, num_numeric*d_model)

    seq_len = tgt.size(0)

    position_ids = torch.arange(seq_len, device=tgt.device).unsqueeze(0)  # shape (1, seq_len)
    pos_embedding = self.positional_embedding(position_ids)               # shape (1, seq_len, d_model * total_fields)

    tgt_embedded = self.dropout(torch.cat([tgt_categorical_embedded, tgt_numeric_embedded], dim=1).unsqueeze(0) + pos_embedding)

    dec_output = tgt_embedded # (batch_size, seq_length, num_features * d_model)
    for dec_layer in self.decoder_layers:
      dec_output = dec_layer(dec_output, tgt_mask)

    # Use last transaction's output to predict the next one
    last_output = dec_output # (batch_size, seq_length, num_features * d_model)
    pred_next_emb = self.embed_pred(last_output) # (batch_size, seq_length, num_features * d_model)

    cat_preds = [head(last_output) for head in self.cat_output_heads]       # logits for each categorical field
    num_preds = [head(last_output) for head in self.num_output_heads]  # numeric values
    return pred_next_emb, cat_preds, num_preds

In [ ]:
# Get training and testing data

# Pad end with 0's
train_num_trans_per_user = []
for user in train_data['cc_num'].unique():
  train_num_trans_per_user.append(train_data[train_data['cc_num'] == user].shape[0])

train_max_len = max(train_num_trans_per_user)

test_num_trans_per_user = []
for user in test_data['cc_num'].unique():
  test_num_trans_per_user.append(test_data[test_data['cc_num'] == user].shape[0])

test_max_len = max(test_num_trans_per_user)

max_len = max(train_max_len, test_max_len)

# Get training data

train_data_by_user = []
for user in train_data['cc_num'].unique():
  if train_data[train_data['cc_num'] == user].shape[0] > 1:
    user_data = train_data[train_data['cc_num'] == user].drop(['cc_num', 'is_fraud'], axis=1)
    user_data = np.append(user_data, np.zeros((max_len - user_data.shape[0], user_data.shape[1])), axis=0)
    train_data_by_user.append(torch.from_numpy(user_data))
train_data_by_user_tensor = torch.stack([user for user in train_data_by_user])

train_tgt_data = train_data_by_user_tensor

# Get testing data

test_data_by_user = []
for user in test_data['cc_num'].unique():
  if test_data[test_data['cc_num'] == user].shape[0] > 1:
    test_user_data = test_data[test_data['cc_num'] == user].drop(['cc_num', 'is_fraud'], axis=1)
    test_user_data = np.append(test_user_data, np.zeros((max_len - test_user_data.shape[0], test_user_data.shape[1])), axis=0)
    test_data_by_user.append(torch.from_numpy(test_user_data))
test_data_by_user_tensor = torch.stack([user for user in test_data_by_user])

test_tgt_data = test_data_by_user_tensor

In [ ]:
print(train_tgt_data.shape)
print(test_tgt_data.shape)

torch.Size([13, 2049, 11])
torch.Size([25, 2049, 11])


In [ ]:
# Hyperparameter tuning
import optuna

# Define objective, then define trial where parameters can vary, then study.optimize

def get_best_params(train_data):
  trial_data = []

  def objective(trial):
    # Define parameters to optimize
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-4, log=True),
        'd_model': trial.suggest_int('d_model', 16, 128),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'num_layers': trial.suggest_int('num_layers', 1, 10),
    }

    # Train model
    np.random.seed(42)

    transformer = Transformer(categorical, numerical, params['num_layers'], max_len+1, params['d_model'] * 4, 1, 4, 128, params['dropout']) # parameters in the custom format for my transformer class
    criterion_cat = nn.CrossEntropyLoss()
    criterion_num = nn.MSELoss()
    optimizer = optim.Adam(transformer.parameters(), lr=params['learning_rate'])

    transformer.train()

    total_loss = 0
    for user in range(len(train_data) // 3):
      optimizer.zero_grad()

      train_input = train_tgt_data[user, :-1, :]  # (seq_len - 1, num_features)

      train_target = train_tgt_data[user, 1:, :]  # (seq_len - 1, num_features)

      _, preds_cat, preds_num = transformer(train_input)

      train_tgt_cat = train_target[:, :len(categorical)].long()  # (seq_len - 1, num_categorical)
      train_tgt_num = train_target[:, len(categorical):len(categorical) + numerical].float()  # (seq_len - 1, num_numeric)

      # Loss calculations
      cat_loss = sum(
          criterion_cat(pred.squeeze(0)[: train_num_trans_per_user[user]-1], train_tgt_cat[:, i][1: train_num_trans_per_user[user]].to(pred.device))
          for i, pred in enumerate(preds_cat)
      ) / len(preds_cat)

      num_loss = sum(
          criterion_num(pred.squeeze(0).squeeze(1)[: train_num_trans_per_user[user]-1], train_tgt_num[:, i][1: train_num_trans_per_user[user]])
          for i, pred in enumerate(preds_num)
      ) / len(preds_num)

      loss = cat_loss * 0.85 + num_loss * 0.15
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    return total_loss / len(train_data)

  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=25, show_progress_bar=True)

  return study.best_params


In [ ]:
categorical = (15, 13, 32, 25, 61, 61, 8) # (category, Month, Day, Hour, Minute, Second, dayOfWeek)
numerical = 4 # (amt, merch_lat, merch_long, LatLong_Dist, hour_amt, category_amt)
num_layers = 6

# best_params = get_best_params(train_data_by_user)
# print(best_params)

In [ ]:
import statistics

# Define transformer model

best_params = {'learning_rate': 1.3446273553256569e-06, 'd_model': 53, 'dropout': 0.3101802514701506, 'num_layers': 8}

# transformer = Transformer(categorical, numerical, num_layers, max_len+1)

transformer = Transformer(categorical, numerical, best_params['num_layers'], max_len+1, best_params['d_model'] * 4, 1, 4, 128, best_params['dropout']) # parameters in the custom format for my transformer class


criterion_cat = nn.CrossEntropyLoss()
criterion_num = nn.MSELoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.0001)

transformer.train()

train_preds_cat = []
train_preds_num = []

for epoch in range(75):
  loss_avg, cat_loss_avg, num_loss_avg = 0, 0, 0
  for user in range(len(train_data_by_user)):
    optimizer.zero_grad()

    train_input = train_tgt_data[user, :-1, :]  # (seq_len - 1, num_features)

    train_target = train_tgt_data[user, 1:, :]  # (seq_len - 1, num_features)

    _, preds_cat, preds_num = transformer(train_input)

    train_tgt_cat = train_target[:, :len(categorical)].long()  # (seq_len - 1, num_categorical)
    train_tgt_num = train_target[:, len(categorical):len(categorical) + numerical].float()  # (seq_len - 1, num_numeric)

    # Loss calculations
    cat_loss = sum(
        criterion_cat(pred.squeeze(0)[: train_num_trans_per_user[user]-1], train_tgt_cat[:, i][1: train_num_trans_per_user[user]].to(pred.device))
        for i, pred in enumerate(preds_cat)
    ) / len(preds_cat)

    num_loss = sum(
        criterion_num(pred.squeeze(0).squeeze(1)[: train_num_trans_per_user[user]-1], train_tgt_num[:, i][1: train_num_trans_per_user[user]])
        for i, pred in enumerate(preds_num)
    ) / len(preds_num)

    loss = cat_loss * 0.85 + num_loss * 0.15
    loss.backward()
    optimizer.step()
    loss_avg += loss.item()
    cat_loss_avg += cat_loss.item()
    num_loss_avg += num_loss.item()
  print('Epoch', epoch+1, 'Loss', loss_avg / len(train_data_by_user), 'Cat loss', cat_loss_avg / len(train_data_by_user), 'Num loss', num_loss_avg / len(train_data_by_user))

Epoch 1 Loss 3.648399994923518 Cat loss 3.2243824005126953 Num loss 6.051165713713719
Epoch 2 Loss 2.9065332412719727 Cat loss 3.173713152225201 Num loss 1.3925133897708013


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
import torch.nn.functional as F

# Get true values
train_tgt_cat = train_tgt_data[-1, :, :][:, :7].long()
train_tgt_num = train_tgt_data[-1, :, :][:, 7:12].float()

all_cat_preds, all_cat_labels = [[] for _ in categorical], [[] for _ in categorical]
all_num_preds, all_num_labels = [[] for _ in range(5)], [[] for _ in range(4)]

# Categorical predictions
for i, pred in enumerate(preds_cat):
  pred_labels = F.softmax(pred.squeeze(0), dim=1).argmax(dim=1).cpu().tolist()
  true_labels = train_tgt_cat[:, i].cpu().tolist()
  all_cat_preds[i].extend(pred_labels)
  all_cat_labels[i].extend(true_labels)

# Numeric predictions
for i, pred in enumerate(preds_num):
  all_num_preds[i].extend(pred.squeeze(0).squeeze(1).cpu().tolist())
  all_num_labels[i].extend(train_tgt_num[:, i].cpu().tolist())

cat_accuracies = [accuracy_score(true[:-1][1: train_num_trans_per_user[-1]], pred[:-1][: train_num_trans_per_user[-1]-1]) for true, pred in zip(all_cat_labels, all_cat_preds)]

num_rmse = [mean_squared_error(true[:-1][1: train_num_trans_per_user[-1]], pred[:-1][: train_num_trans_per_user[-1]-1]) for true, pred in zip(all_num_labels, all_num_preds)]

categories = ['Category', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'dayOfWeek']
for i in range(7):
  print(categories[i], 'accuracy:', str(cat_accuracies[i] * 100) + '%')

nums = ['Amount', 'Merch Lat', 'Merch Long', 'LatLong_Dist']
for i in range(4):
  print(nums[i], 'MSE:', str(num_rmse[i]))

In [ ]:
import statistics
print('Categorical predictions')
for i, pred in enumerate(preds_cat):
  print('PRED', F.softmax(pred.squeeze(0), dim=1)[: train_num_trans_per_user[-1]-1].argmax(dim=1).cpu().tolist())
  print('TRUE', train_tgt_cat[:, i][1: train_num_trans_per_user[-1]].cpu().tolist())
  print(" ")
print('Numerical predictions')
for i, pred in enumerate(preds_num):
  print('PRED', pred.squeeze(0)[: train_num_trans_per_user[-1]-1].squeeze(1).cpu().tolist())
  print('TRUE', train_tgt_num[:, i][1: train_num_trans_per_user[-1]].cpu().tolist())
  print(" ")

### Evaluate Model on Test Data

In [ ]:
test_preds_cat = [] # List of 25 lists of probabilities for each of the 7 categorical features (1, 2048, 15), (1, 2048, 13), etc for all the transactions of a single user
test_preds_num = [] # List of 25 lists of predictions for each of the 4 numeric features (1, 2048, 1) for all the transactions of a single user
transformer.eval()
with torch.no_grad():
  for user in range(len(test_tgt_data)):
    pred_next_emb, preds_cat, preds_num = transformer(test_tgt_data[user, :-1, :])
    test_preds_cat.append(preds_cat)
    test_preds_num.append(preds_num)

In [ ]:
import torch.nn.functional as F

preds_cat_tensor = []
for user in range(len(test_preds_cat)): # want to make a tensor with 15 columns and ~25 * 2048 rows for the categorical probabilities for all users
  preds_for_user = test_preds_cat[user]
  preds_for_user_concat = torch.cat([F.softmax(pred, dim=2).argmax(dim=2).cpu() for pred in preds_for_user], dim=0) # (2048 rows, 4 columns)
  preds_for_user_concat = torch.transpose(preds_for_user_concat, 0, 1)[:test_num_trans_per_user[user], :]
  preds_cat_tensor.append(preds_for_user_concat)

preds_cat_tensor = torch.concat(preds_cat_tensor, dim=0)
print(preds_cat_tensor.shape)

In [ ]:
preds_num_tensor = []
for user in range(len(test_preds_num)): # want to make a tensor with 4 columns and ~25 * 2048 rows for the numerical predictions for all users
  preds_for_user = test_preds_num[user]
  preds_for_user_concat = torch.cat([pred.squeeze(0)[:test_num_trans_per_user[user], :] for pred in preds_for_user], dim=1) # (2048 rows, 4 columns)
  preds_num_tensor.append(preds_for_user_concat)

preds_num_tensor = torch.concat(preds_num_tensor, dim=0)
print(preds_num_tensor.shape)

In [ ]:
preds_tensor = torch.cat([preds_cat_tensor, preds_num_tensor], dim=1)
print(preds_tensor.shape)

In [ ]:
preds_df = pd.DataFrame(preds_tensor.numpy(), columns=test_data.columns.drop(['cc_num', 'is_fraud']))

preds_df['is_fraud'] = test_data.reset_index(drop=True)['is_fraud']
preds_df['cc_num'] = test_data.reset_index(drop=True)['cc_num']

In [ ]:
preds_df.head()

In [ ]:
torch.save(transformer.state_dict(), '/content/drive/MyDrive/transformer_model.pth')

In [ ]:
test_data.to_csv('/content/drive/MyDrive/test_data_transformer.csv', index=False)

In [ ]:
train_data.to_csv('/content/drive/MyDrive/train_data_transformer.csv', index=False)

In [ ]:
preds_df.to_csv('/content/drive/MyDrive/preds_df_transformer.csv', index=False)